Item 44 Use Plain Attributes Instead of Setter and Getter Methods  

Things to Remember
- Define new class interfaces using simple public attributes and avoid defining setter and getter methods
- Use @property to define special behavior when attributes are accessed on your objects, if necessary.
- Follow the rule of least surprise and avoid odd side effects in your @property methods.
- Ensure that @property methods are fast; for slow or complex work-especially involving I/O or causing side effects-use normal methods instead.   

In [ ]:
# - we coming from other languages may naturally 
#   try to implement explicit getter and setter
#   methods in our classes

class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms
    
    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms       

In [ ]:
# - using these setters and getter is simple
#   but not Pythonic
r0 = OldResistor(50e3)
print('Before:', r0.get_ohms())
r0.set_ohms(10e3)
print('After: ', r0.get_ohms())

In [ ]:
# - and then it become clumsy for
#   operations like incrementing
#   in place
r0.set_ohms(r0.get_ohms() - 4e3)
assert r0.get_ohms() == 6e3

The Pythonic way
- do not implement explicit setter or getter methods
- should always start your implementation with simple public attributes
- use @property decorator if you need special behavior on an attribute

In [ ]:
# start with public attributes
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0


In [ ]:
r1 = Resistor(50e3)
r1.ohms = 10e3
r1.ohms += 5e3 # looks natural and clear now

In [ ]:
# - next I need special behavior
# - I want to vary the current by 
#   assigning the voltage property

class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    @property
    # - the name must match
    #   the intended property
    #   name (voltage in this case)
    def voltage(self): 
        return self._voltage
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms       

In [ ]:
r2 = VoltageResistance(1e3)
print(f'Before: {r2.current:.2f} amps')
r2.voltage = 10 # call the setter
print(f'After:  {r2.current:.2f} amps')

In [ ]:
# - perform type checking and validation on
#   values passed to the class

class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms    

In [ ]:
r3 = BoundedResistance(1e3)
r3.ohms = 0 

How the validation logic gets executed?
- BoundedResistance.\__init__ calls Resistor.\__init\__
- Resistor.\__init\__ assigns self._ohms = 0
- this assignment causes the setter from BoundedResistance to be called
- the validation logic in the setter is executed  

In [ ]:
# - use @property to make attributes from parent
#   classes immutable
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Ohms is immutable")
        self._ohms = ohms

In [ ]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3 # error

In [ ]:
# - don't set other attributes in getter
#   methods to make their behavior is
#   not surprising
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        # - set other attributes while reading ohms
        # - this is bad as it would lead to extremely
        #   bizzare behavior 
        self.voltage = self._ohms * self.current
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms    

In [ ]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print(f'Before: {r7.voltage:.2f}')
# - I just want to know the value of ohms
# - yet I have no idea by doing so I also
#   changed the value of voltage at the 
#   same time
r7.ohms
print(f'After:  {r7.voltage:.2f}')


Best Policy in using @property
- Modify only related object state in @property.setter methods
- Avoid other side effects that the caller may not expect beyond the object such as
    - importing modules dynamically
    - running slow helper functions
    - doing I/O
    - making expensive database queries
- Users of a class will expect its attributes to be quick and easy to access.  

Shortcoming of @property
- Can only be shared by subclasses
- Unrelated classes can't share the same implementation

Descriptor 
- Check Item 46 for how descriptors enable reusable property logic and many more. 